# Build Dataset + Index (offline)

Этот ноутбук делает первичную загрузку датасета и построение индекса **вне** сессии агента.
Далее `index_id` нужно вставить в `configs/search_dialog_chat_*.json`.


In [ ]:
import json
import sys
from pathlib import Path
import yaml

ROOT = Path('..').resolve()
SRC = (ROOT / 'src').resolve()
sys.path.insert(0, str(SRC))

from gigasearch import GigaSearchClient
from agent.settings import load_yaml, load_json

cfg = load_yaml(str(ROOT / 'configs' / 'indexing.yaml'))
cfg


In [ ]:
gs = cfg.get('gigasearch') or {}
cert_cfg = (gs.get('cert') or {})

client = GigaSearchClient(
    base_url=str(gs.get('base_url', '')).rstrip('/'),
    source_uuid=str(gs.get('source_uuid', '')),
    cert=(str(cert_cfg.get('crt_path', '')), str(cert_cfg.get('key_path', ''))),
    verify_ssl=bool(gs.get('verify_ssl', False)),
    timeout_sec=int(gs.get('timeout_sec', 600)),
)

paths = cfg.get('paths') or {}
de_cfg = load_yaml(str(paths.get('data_enrichment_config')))
indexer_cfg = load_json(str(paths.get('indexer_config')))

index_section = cfg.get('index') or {}
index_id = index_section.get('index_id') or None

dataset_uuid = client.create_dataset(de_cfg)
dataset_uuid


In [ ]:
uploads = cfg.get('upload') or []
for u in uploads:
    client.upload_files(dataset_uuid, u['path'], product_name=u['product_name'])

client.save_dataset_version(dataset_uuid)
last_ver = client.wait_last_version_done(dataset_uuid)
last_ver


In [ ]:
index_uuid = client.create_index(dataset_uuid, indexer_cfg, index_id=index_id)
client.update_index(index_uuid)
index_status = client.wait_index_done(index_uuid)

final_index_id = index_status.get('index_id') or index_id or indexer_cfg.get('index_id')
print('dataset_uuid:', dataset_uuid)
print('index_uuid  :', index_uuid)
print('index_id    :', final_index_id)
